In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 读取数据集
mnist = input_data.read_data_sets('./datas/MNIST_data/', one_hot=True)

print('训练集量', mnist.train.num_examples)
print('验证集量', mnist.validation.num_examples)
print('测试集量', mnist.test.num_examples)
print('测试集标签', mnist.test.labels)

W1111 12:28:56.716520 26480 deprecation.py:323] From <ipython-input-2-3f00b2638b94>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1111 12:28:56.716520 26480 deprecation.py:323] From c:\users\86132\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1111 12:28:56.716520 26480 deprecation.py:323] From c:\users\86132\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and wi

Extracting ./datas/MNIST_data/train-images-idx3-ubyte.gz


W1111 12:28:57.122673 26480 deprecation.py:323] From c:\users\86132\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1111 12:28:57.138297 26480 deprecation.py:323] From c:\users\86132\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1111 12:28:57.232023 26480 deprecation.py:323] From c:\users\86132\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from te

Extracting ./datas/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./datas/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./datas/MNIST_data/t10k-labels-idx1-ubyte.gz
训练集量 55000
验证集量 5000
测试集量 10000
测试集标签 [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
# 相关参数设置
INPUT_NODE = 784     # 输入层节点数，图片像素的一维矩阵 28*28=784
OUTPUT_NODE = 10     # 输出节点数，类别数目

# 神经网络参数
LAYER1_NODE = 500    # 隐藏层节点数，仅一个隐藏层
BATCH_SIZE = 100     # 每个batch 的训练个数

LEARNING_RATE_BASE = 0.8       # 基础学习率
LEARNING_RATE_DECAY = 0.99     # 学习率衰减
REGULARIZATION_RATE = 0.0001   # 正则化项中损失函数的系数
TRAINING_STEPS = 30000         # 戌年轮数
MOVING_AVERAGE_DECAY = 0.99

#
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 当没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class == None:
        # 计算隐藏层的前向传播结果，使用ReLU激活函数
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        
        # 计算输出层的前向传播结果，因为计算损失函数时会计算softmax函数，
        # 所以这里不加入损失函数,
        return tf.matmul(layer1, weights2) + biases2
    
    else:
        # 计算出变量的滑动平均值
        # 然后再计算神经网络相应的先前传播结果
        layer1 = tf.nn.relu(
            tf.matmul(input_tensor, avg_class.average(weights1)) +
            avg_class.average(biases1)
        )
        
        return tf.matmul(
            layer1, avg_class.average(weights2) + avg_class.average(biases2))

    
def train(mnist):
    # 训练集
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    # 隐藏层参数：权重
    weights1 = tf.Variable(
        # 从截断的正态分布中输出随机值, 行数等于输入节点数，列数等于下一层节点数
        tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1)    
    )
    # 隐藏层参数：偏置项，维度等于节点数
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    
    # 输出层参数：权重
    weights2 = tf.Variable(
        tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1)  
    )
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

    
    # 计算当前参数下神将网络向前的传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义训练轮数的变量， 将这个变量指定为不可训练变量
    global_step = tf.Variable(0, trainable=False)
    # 初始化滑动平均类
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY, global_step)
    
    # 在所有神经网络的参数变量上，使用滑动平均，辅助变量不需要，
    # 返回所有没有指定 trainable=False 的变量集合
    variable_averages_op = variable_averages.apply(
        tf.trainable_variables())
    
    # 计算使用滑动平均之后的向前传播的结果。
    average_y = inference(
        x, variable_averages, weights1, biases1, weights2, biases2)

    # 计算交叉熵作为刻画预测值和真实值之间差距的损失函数，当分类只有一个正确答案时
    # 可以使用sparse_softmax_cross_entropy_with_logits 函数来加速交叉熵的计算
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=y, labels=tf.argmax(y_, 1))
    # 计算当前batch中的所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)

    # L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 计算模型正则化损失，一般只计算神经网络边上权重的正则化
    regularization = regularizer(weights1) + regularizer(weights2)
    # 总损失等于交叉熵损失和正则化损失的和
    loss = cross_entropy_mean + regularization

    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,    # 基础学习率，随着迭代的进行，更新变量
        global_step,           # 当前迭代的轮数
        mnist.train.num_examples / BATCH_SIZE,   # 过完所有训练数据需要的轮数
        LEARNING_RATE_DECAY    # 学习率衰减速度
    )

    
    # 损失函数优化,包含交叉熵损失和正则化损失
    train_step = tf.train.GradientDescentOptimizer(
        learning_rate).minimize(loss, global_step=global_step)

    # 通过反向传播更新神经网络中的参数。并更新每个参数的滑动平均值
    # train_op = tf.group(train_step, variable_averages_op) 等价于
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')


    # 判断预测结果和真实结果是否相等
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    # 将比较结果（bool）转为数值，计算预测的准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with tf.Session() as sess:
        #  初始化所有变量
        tf.global_variables_initializer().run()
        
        
        # 验证数据
        validate_feed = {
            x: mnist.validation.images,
            y_: mnist.validation.labels
        }
        # 测试数据
        test_feed  = {
            x: mnist.test.images,
            y_: mnist.test.labels
        }

        # 迭代神经网络
        # for i in range(TRAINING_STEPS):
        for i in range(2000):
            if i%1000 == 0:
                # 每隔1000次计算验证集数据结果，
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print(f'经过{i}轮训练后，模型的准确率是{validate_acc}')
            
            # 训练模型
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})

        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print(f'模型训练完成，测试集的准确率是{test_acc}')

    
def main(argv=None):
    mnist = input_data.read_data_sets('./datas/MNIST_data/', one_hot=True)
    train(mnist)
    

if __name__ == '__main__':
    tf.app.run()

Extracting ./datas/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./datas/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./datas/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./datas/MNIST_data/t10k-labels-idx1-ubyte.gz
经过0轮训练后，模型的准确率是0.14820000529289246
经过1000轮训练后，模型的准确率是0.09239999949932098
模型训练完成，测试集的准确率是0.09740000218153


SystemExit: 